# Pytorch Tutorial - Autograd & MLP (Multi-layer perceptron)

### Autograd
- autograd 패키지는 텐서의 모든 연산에 대한 자동 미분을 제공
- 실행-기반-정의(define-by-run)프레임워크로, 코드를 어떻게 작성하여 실행하는냐에 따라 역전파가 정의된다는 것을 의미
- 역전파는 학습 과정의 매 단계마다 달라짐

### Tensor
- `torch.Tensor` 클래스의 `.requires_grad`속성을 `True`로 설정하면, 해당 텐서에서 이루어진 모든 연산을 추적(track)하기 시작
- 계산이 완료된 후 `.backward()`를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있으며 이 Tensor의 변화도(gradient)는 `.grad` 속성에 누적됨
- Tensor가 기록을 추적하는 걸 중단하려면 `.detach()`를 호출하여 연산기록으로부터 분리하여 연산이 추적되는 것을 방지할 수 있음
- 기록을 추적하는 것(과 메모리를 사용하는 것)을 방지하기 위해서 코드 블럭을 `with.torch.no_grad():`로 감쌀 수 있음
- 이는 변화도는(gradient)는 필요없지만 `requires_grad=True`가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evaluate)할 때 유용
- Autograd 구현에서 `Function` 클래스는 매우 중요한 역할을 수행
- `Tensor`와 `Function`은 서로 연결되어 있고 모든 연산과정을 부호화하여 순환하지 않는 그래프 생성
- 각 Tensor는 `.grad_fn`속성을 가지고 있는데 이는 `Tensor`를 생성한 `Function`을 참조(단, 사용자가 만든 Tensor는 예외이며, 사용자가 만든 Tensor가 아닌 연산에 의해 생긴 텐서와 같은 경우는 모두 `Function`을 참조)
- 도함수를 계산하기 위해서는 `Tensor`의 `.backward()` 호출

In [30]:
import torch

In [31]:
# x의 연산 과적을 추적하기 위해 requires_grad=True로 설정
x = torch.ones(2, 2, requires_grad=True)
print(x)

# 직접 생선한 Tensor이기 때문에 grad_fn이 None인 것을 확인할 수 있음
#grad_fn속성 : Tensor를 생성한 Function 참조
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


In [32]:
# y, z, out은 연산의 결과로 생성된 것이기 때문에 grad_fn을 갖고 있는 것을 확인 가능
y = x + 2
y.retain_grad()
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [33]:
# 연산의 결과로 생성된 것이기 때문에 grad_fn을 갖는 것을 확인 가능
print(y.grad_fn)

In [34]:
z = y * y * 3
out = z.mean()

# 각각 사용한 func에 맞게 grad_fn이 생성된 것을 확인할 수 있음
print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


- `requires_grad_()`를 사용하면 기존 Tensor의 `requires_grad`값을 바꿀 수 있음
- 입력 값이 지정되지 않으면 기본 값은 `False`

In [35]:
a=torch.randn(2,2)
print(a)

tensor([[ 1.0235,  0.0437],
        [-1.4578, -0.6392]])


In [36]:
a = ((a * 3) / (a - 1))
print(a)
print(a.requires_grad)

tensor([[130.5737,  -0.1370],
        [  1.7794,   1.1698]])
False


In [37]:
#requires_grad_()를 사용하면 requires_grad값을 바꿀 수 있음
a.requires_grad_(True)

tensor([[130.5737,  -0.1370],
        [  1.7794,   1.1698]], requires_grad=True)

In [38]:
print(a.requires_grad)

True


In [39]:
#속성을 상속받음
b=(a*a).sum()
print(b)
print(b.requires_grad)

tensor(17054.0449, grad_fn=<SumBackward0>)
True


변화도(gradient)

In [ ]:
print(out)

#이전에 만든 out을 사용해서 역전파 진행
# 여러 번 역전파를 하기 위해서는 retain_graph=True로 지정
out.backward(retain_graph=True)
# out.backward(torch.tensor(1.))을 진행하는 것과 동일

In [ ]:
# 역전파를 진행한 후의 x값
# d(out) / dx
print(x.grad)
print(y.grad)
print(z.is_leaf)

In [ ]:
#이전에 만든 out을 사용해서 역전파 진행

#retain_grad() : leaf가 아닌 Tensor의 .grad속성 활성화
#중간값에 대한 미분값을 보고 싶다면 해당 값에 대한 retain_grad()호출해야함
y.retain_grad()
x.retain_grad()
z.retain_grad()
#backward() : 모든 변화도(gradient)를 자동으로 계산
#여러 번 미분을 진행하기 위해서는 retain_graph=True로 설정해줘야함(안 하면 에러)
out.backward(retain_graph=True)

In [40]:
print(out)

#이전에 만든 out을 사용해서 역전파 진행

y.retain_grad()     #중간값에 대한 미분값을 보고 싶다면 해당 값에 대한 retain_grad()호출해야함
z.retain_grad()
out.backward()      #여러 번 미분을 진행하기 위해서는 retain_graph=True로 설정해줘야함(안 하면 에러)

# out.backward(torch.tensor(1.))을 진행하는 것과 동일
print(x.grad)
print(y.grad)
print(z.grad)
print(z.is_leaf)

out.backward()
print(x.grad)
print(y.grad)

tensor(27., grad_fn=<MeanBackward0>)
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
tensor([[0.2500, 0.2500],
        [0.2500, 0.2500]])
False


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
x = torch.ones(2, 2, requires_grad=True)  # 요소값이 1인 2x2행렬을 선언한다.
y = x + 2
z = y * y * 3
out = z.mean()

print(out)

y.retain_grad()
out.backward(retain_graph=True)

print(x.grad)
print(y.grad)
print(z.grad)       # z.retain_grad()를 호출하지 않으면 grad값을 저장하지 않기 때문에 grad 속성을 볼 수 있음
print(z.is_leaf)

out.backward()
print(x.grad)
print(y.grad)

In [ ]:
print(x.grad)
print(y.grad)
print(z.grad)

- 일반적으로 `torch.autograd`는 벡터-야코비안 곱을 계산하는 엔진
- `torch.autograd`를 사용하면 전체 야코비안을 직접 계산할 수는 없지만, 벡터-야코비안 곱은 `backward`에 해당 벡터를 인자로 제공하여 얻을 수 있음

In [41]:
x = torch.randn(3, requires_grad=True)
print(x)
y = x * 2
print(y)

#.data.norm() : L2 norm(Frobenius norm)
#=sqrt(sum(y의 모든 요소^2))
while y.data.norm() < 1000:
  y = y * 2

print(y)

tensor([-0.7879, -2.0423, -0.7464], requires_grad=True)
tensor([-1.5758, -4.0846, -1.4928], grad_fn=<MulBackward0>)
tensor([ -403.4163, -1045.6548,  -382.1657], grad_fn=<MulBackward0>)


In [42]:
# scalar 값이 아닌 y의 벡터-야코비안 곱을 구하는 과정
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


- `with torch.no_grad()`로 코드 블록을 감싸서 `autograd`가 `.requires_grad=True`인 Tensor의 연산 기록을 추적하는 것을 멈출 수도 있음

In [43]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print(x.requires_grad)

True
True
True


- 또는 .detach()를 호출하여 내용물은 같지만 requires_grad가 다른 새로운 Tensor를 생성할 수도 있음

In [ ]:
print(x.requires_grad)
y=x.detach()
print(y.requires_grad)
print(x.eq(y).all())

### ANN(Artificial Neural Networks)
- 신경망은 `torch.nn`패키지를 사용하여 생성가능
- nn은 모델을 정의하고 미분하기 위해서 위에서 살펴본 `autograd`를 사용
- `nn.Module`은 layer와 `output`을 반환하는 `forward(input)`메소드 포함
예제)

- 간단한 순전파 네트워크(feed-forward-network)
- 입력을 받아 여러 계층(layer)에 차례로 전달 후 최종 출력 제공
- 신경망의 일반적인 학습 과정

1. 학습 가능한 매개변수(가중치)를 갖는 신경망 정의
2. 데이터 셋 입력 반복
3. 입력을 신경망에서 전파(process)
4. 손실(loss, 입력값-예측값)를 계산
5. 변화도(gradient)를 신경망의 매개변수들에 의해 역으로 전파(역전파)
6. 신경망의 가중치 갱신
- 새로운 가중치(weight)=가중치(weight)-학습률(learning rate)*변화도(gradient)

In [44]:
#라이브러리 import
import pandas as pd

from sklearn.datasets import load_iris

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [45]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer0 = nn.Linear(4, 128)
        self.layer1 = nn.Linear(128, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.layer4 = nn.Linear(16, 3)

        self.bn0 = nn.BatchNorm1d(128)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)

        self.act = nn.ReLU()

    def forward(self, x):
      x = self.act(self.bn0(self.layer0(x)))
      x = self.act(self.bn1(self.layer1(x)))
      x = self.act(self.bn2(self.layer2(x)))
      x = self.act(self.layer3(x))
      x = self.layer4(x)

      return x

### 손실함수(Loss function)
- (output, target)을 한 쌍으로 입력받아 출력이 정답으로부터 얼마나 떨어져있는지 계산
- `forward()`만 정의하면 `backward()`는 `autograd`에 의해 자동 정의
- 모델의 학습 가능한 매개변수는 `net.parameters()`에 의해 변환

In [46]:
# 랜덤 값 생성
criterion = nn.CrossEntropyLoss()

ex_X, ex_y = torch.randn([4, 4]), torch.tensor([1, 0, 2, 0])
print('ex_X : ', ex_X)
print('ex_y : ', ex_y)

#모델 정의
net = Net()
#입력값을 모델로 훈련시킨 후 나온 출력값
output = net(ex_X)
print(output)
#손실계산
loss = criterion(output, ex_y)
#계산한 손실 출력
print('loss: ', loss.item())
  
net.zero_grad()

print('layer0.bias.grad before backward')
print(net.layer4.bias.grad)

print(net.layer4.bias.is_leaf)

loss.backward()

print('layer0.bias.grad after backward')
print(net.layer4.bias.grad)

ex_X :  tensor([[ 0.9859,  0.0335,  0.0845, -2.0727],
        [-0.2647,  0.2539, -0.8186, -0.9914],
        [ 0.2767, -0.8893, -0.3133, -1.4691],
        [-2.3311, -0.3446, -0.2750,  1.3268]])
ex_y :  tensor([1, 0, 2, 0])
tensor([[-0.0179,  0.1466, -0.2009],
        [-0.0852,  0.0872, -0.2477],
        [ 0.0134,  0.3783, -0.3096],
        [-0.2629,  0.3525,  0.0577]], grad_fn=<AddmmBackward0>)
loss:  1.2414988279342651
layer0.bias.grad before backward
None
True
layer0.bias.grad after backward
tensor([-0.1966,  0.1688,  0.0277])


In [47]:
#net.parameters()에는 각각의 층들마다의 weight, bias값들이 들어있음
#Linear Layer : 4, 128, 64, 32, 16, 3 -> 6개*2(wieght, bias) + BatchNorm1d Layer : 128, 64, 32->3개*2(weight, bias)
params = list(net.parameters())
print(len(params))
print(params[0].size())   # layer0의 weight

# for i in range(len(params)):
#   print(params[i].size())

16
torch.Size([128, 4])


### 가중치 갱신
가장 단순한 갱신 규칙 : 확률적 경사 하강법(SGD, Stochastic Gradient Descent)

- 새로운 가중치(weight)=가중치(weight)-학습률(learning rate)*변화도(gradient)

In [48]:
# torch.optim 패키지에 다양한 갱신 규칙이 규현되어 있음

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.001)

optimizer.zero_grad()
output = net(ex_X)
loss = criterion(output, ex_y)
loss.backward()
optimizer.step()  # 업데이트 진행

MLP 모델

In [49]:
#data load
dataset = load_iris()

data = dataset.data
label = dataset.target

print(dataset.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [52]:
print('shape of data: ', data.shape)
print('shape of label: ', label.shape)

shape of data:  (150, 4)
shape of label:  (150,)


In [53]:
# 훈련과 테스트 데이터로 나누기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.25)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25)
print(len(X_train))
print(len(X_test))

84
38


In [54]:
# DataLoader 생성
# numpy->Tensor
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

train_set = TensorDataset(X_train, y_train)

train_loader = DataLoader(train_set, batch_size=4, shuffle=True)

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer0 = nn.Linear(4, 128)
        self.layer1 = nn.Linear(128, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.layer4 = nn.Linear(16, 3)

        self.bn0 = nn.BatchNorm1d(128)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(32)

        self.act = nn.ReLU()

    def forward(self, x):
      x = self.act(self.bn0(self.layer0(x)))
      x = self.act(self.bn1(self.layer1(x)))
      x = self.act(self.bn2(self.layer2(x)))
      x = self.act(self.layer3(x))
      x = self.layer4(x)

      return x
      # return nn.Softmax(x)

In [ ]:
net = Net()
print(net)

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
epochs = 200

In [ ]:
losses = list()
accuracies = list()

for epoch in range(epochs):
  epoch_loss = 0  
  epoch_accuracy = 0
  for X, y in train_loader:
  
    optimizer.zero_grad()
    
    output = net(X)

    loss = criterion(output, y)
    loss.backward()
    
    optimizer.step()
    # output = [0.11, 0.5, 0.8] --> 예측 클래스 값
    _, predicted = torch.max(output, dim=1)
    accuracy = (predicted == y).sum().item()
    epoch_loss += loss.item()
    epoch_accuracy += accuracy

  epoch_loss /= len(train_loader)
  epoch_accuracy /= len(X_train)
  print("epoch :{}, \tloss :{}, \taccuracy :{}".format(str(epoch+1).zfill(3),round(epoch_loss,4), round(epoch_accuracy,4)))
  
  losses.append(epoch_loss)
  accuracies.append(epoch_accuracy)

In [ ]:
# Plot result

import matplotlib.pyplot as plt

plt.figure(figsize=(20,5))
plt.subplots_adjust(wspace=0.2)

plt.subplot(1,2,1)
plt.title("$loss$",fontsize = 18)
plt.plot(losses)
plt.grid()
plt.xlabel("$epochs$", fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)


plt.subplot(1,2,2)
plt.title("$accuracy$", fontsize = 18)
plt.plot(accuracies)
plt.grid()
plt.xlabel("$epochs$", fontsize = 16)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)

plt.show()

In [ ]:
# Test

output = net(X_test)
print(torch.max(output, dim=1))
_, predicted = torch.max(output, dim=1)
accuracy = round((predicted == y_test).sum().item() / len(y_test),4)

print("test_set accuracy :", round(accuracy,4))

### Tensorflow-Tutorial(GradientTape, MLP)
그래디언트 테이프

- 자동미분을 위한 `tf.GradientTape` API제공
- `tf.GradientTape`는 모든 연산을 테이프에 기록
- 후진 방식 자동 미분(reverse mode differetiation)을 사용해서 테이프에 기록된 연산 그래디언트 계산

In [ ]:
import tensorflow as tf

print(tf.__version__)

In [ ]:
x = tf.ones((2, 2))
# 1, 1
# 1, 1
with tf.GradientTape() as t:
  t.watch(x)
  #reduce_sum() : 특정 축을 기준으로 합을 구해줌, 아무것도 입력 안 하면 모든 요소의 합
  y = tf.reduce_sum(x)
  print('y: ', y)
  z = tf.multiply(y, y)
  print('z: ', z)

# 입력 텐서 x에 대한 z의 도함수
dz_dx = t.gradient(z, x)
print(dz_dx)

for i in [0, 1]:
  for j in [0, 1]:
    # dz_dx[i][j]가 8이 아니면 AssertionError
    assert dz_dx[i][j].numpy() == 8.0

In [ ]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    print('y: ', y)
    z = tf.multiply(y,y)
    print('z: ', z)
    
# 입력 텐서 x에 대한 z의 도함수
dz_dx = t.gradient(z, x)
print(dz_dx)
for i in [0, 1]:
    for j in [0, 1]:
        # AssertionError가 발생하지 앟음
        assert dz_dx[i][j].numpy() == 8.0

In [ ]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y,y)
    
# tf.GradientTape() 안에서 계산된 중간 값에 대한 그래디언트도 구할 수 있습니다.
# 테이프 사용하여 중간값 y에 대한 도함수를 계산합니다.
dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0

- `GradientTape.gradient()` 메소드가 호출되면 GradientTape에 포함된 리소스가 해제
- 동일한 연산에 대해 여러 gradient를 계산하려면 지속성있는(`persistent=True`) 그래디언트 테이프 생성하면됨
- 이렇게 생성한 그래디언트 테이프는 `gradient()`메소드의 다중 호출 허용

In [ ]:
x = tf.constant(3.0)
print(x)
with tf.GradientTape(persistent=True) as t:
  t.watch(x)
  y = x * x
  z = y * y.  # z = x ^ 4
dz_dx = t.gradient(z, x)  # 108.0 (4*x^3 at x = 3)
print(dz_dx)
dy_dx = t.gradient(y, x)  # 6.0   (2 * x at x = 3)
print(dy_dx)
del t  # 테이프에 대한 참조를 삭제합니다.

### 제어 흐름 기록
- 연산이 실행되는 순서대로 테이프에 기록되기 때문에, 파이썬 제어흐름이 자연스럽게 처리됨

In [ ]:
def f(x, y):
  output = 1.0
  for i in range(y):
    if i > 1 and i < 5:   # output(1) * 2 * 3 * 4    #i가 2,3,4일 때 output*=x output은 x^n승
      output = tf.multiply(output, x)
  return output

def grad(x, y):
  with tf.GradientTape() as t:
    t.watch(x)
    out = f(x, y)
  return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)
print(x)

print(grad(x, 6).numpy())	#output=2^3->gradient=3*2^2
assert grad(x, 6).numpy() == 12.0

print(grad(x, 5).numpy())	#output=2^3->gradient=3*2^2
assert grad(x, 5).numpy() == 12.0

print(grad(x, 4).numpy()) 	#output=2^2->gradient=2*2^1
assert grad(x, 4).numpy() == 4.0

### 고계도(Higher-order) 그래디언트
- `GradientTape` 컨텍스트 매이저 안에 있는 연산들은 자동미분을 위해 기록됨
- 만약 이 컨텍스트 안에서 그래디언트를 계산하면 해당 그래디언트 연산 또한 기록됨

In [ ]:
x = tf.Variable(1.0) 
print(x)

with tf.GradientTape() as t:
  with tf.GradientTape() as t2:
    y = x * x * x
    # 't' 컨텍스트 매니저 안의 그래디언트를 계산합니다.
    # 이것은 또한 그래디언트 연산 자체도 미분가능하다는 것을 의미합니다.
  dy_dx = t2.gradient(y, x)      # dy_dx = 3 * x^2 at x = 1
d2y_dx2 = t.gradient(dy_dx, x)   # d2y_dx2 = 6 * x at x = 1

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0
print(dy_dx)
print(d2y_dx2)

### ANN(Artificial Neural Network)
Sequential 모델을 사용하는 경우

- `Seuquential` 모델은 각 레이어에 정확히 하나의 입력 Tensor와 하나의 출력 Tensor가 있는 일반 레이어 스택에 적합

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),  # Pytorch - nn.Linear 
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
# [1, 1, 1] --> [o, o] --> [o, o, o] --> [o, o, o, o]
y = model(x)
print(y)

In [ ]:
# Input --> model1 --> model2 --> output
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),  # Pytorch - nn.Linear
        # dense --> act:relu
        layers.Dense(3, activation="relu", name="layer2"),
        # dense --> act:relu
        layers.Dense(4, name="layers3"),
        # dense
    ]
    
input
model1 = model
model2 = model
output

input
dense(2)
dense(3)
dense(4)
dense(2)

)

In [ ]:
# Create 3 layers
# 위의 함수와 동일
# Sequential 함수를 사용하지 않고 쌓을 경우
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))
print(y)    # y = w * x

In [ ]:
# layers 속성을 사용해서 레이어에 대해 접근할 수 있음
model.layers

In [ ]:
# add() 메서드를 통해서 Sequential 모델을 점진적으로 작성할 수도 있음
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

In [ ]:
model.layers

In [ ]:
# pop() 메서드를 사용하면 레이어를 제거할 수 있음
model.pop()
print(len(model.layers))    #2

### 패션 MNIST를 사용한 분류 문제
10개의 카테고리, 70000개의 흑백이미지(28*28)

훈련에 60000개, 평가에 10000개 사용

In [ ]:
# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

# 헬퍼(helper) 라이브러리를 임포트합니다
import numpy as np
import matplotlib.pyplot as plt

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

데이터 전처리

In [ ]:
# 신경망 모델에 주입하기 전에 값의 범위를 0~1로 조정
train_images = train_images / 255.0
test_images = test_images / 255.0

모델구성

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
model.compile(optimizer='adam', # SGD, SGD + momentum
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5)

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print("Test loss:", test_loss)
print("Test accuracy:", test_acc)

In [ ]:
# 훈련된 모델을 사용하여 이미지에 대한 예측 만들기
predictions = model.predict(test_images)
# 테스트 세트에 있는 각 이미지에 대한 예측을 진행한 후, 첫번째 예측 값
# 10개의 옷 품목에 상응하는 모델의 신뢰도(confidence)를 나타냄
predictions[0]

In [ ]:
# 가장 높은 신뢰도를 가진 레이블 출력
print(np.argmax(predictions[0]))
# 실제 테스트 데이터의 0번째 값
print(test_labels[0])

In [ ]:
# 10개의 클래스에 대한 예측을 모두 그래프로 표현
# 올바르게 예측된 레이블은 파란색으로, 잘못 예측된 레이블은 빨강색으로 표현
# 숫자는 예측 레이블의 신뢰도 퍼센트
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
# 처음 X 개의 테스트 이미지와 예측 레이블, 진짜 레이블을 출력합니다
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)
plt.show()